In [1]:
!pip install openrouteservice pandas networkx folium


In [2]:
locations = {
    "Farm_Punjab": [75.8573, 30.9000],        # Ludhiana
    "Farm_Haryana": [76.9905, 28.4595],       # Gurugram
    "Farm_UP": [80.9462, 26.8467],            # Lucknow
    "Warehouse_Punjab": [75.3433, 31.3260],   # Jalandhar
    "Warehouse_Haryana": [76.3869, 30.7333],  # Ambala
    "Retailer_Delhi": [77.1025, 28.7041],     # Delhi
    "Retailer_UP": [78.0322, 27.1767],        # Agra
    "Retailer_Punjab": [75.7885, 31.6340]     # Amritsar
}


In [4]:
import openrouteservice
import pandas as pd
import networkx as nx

# Initialize ORS client
client = openrouteservice.Client( key= '5b3ce3597851110001cf6248c499806c1caa4c3f9b1a288cc319b000')  # Replace with your API key

# Convert to list of coordinates
coords = list(locations.values())
names = list(locations.keys())

# Request distance & time matrix
matrix = client.distance_matrix(
    locations=coords,
    metrics=["distance", "duration"],
    resolve_locations=True,
    units="km"
)

# Build DataFrame for visualization
dist_df = pd.DataFrame(matrix['distances'], index=names, columns=names)
time_df = pd.DataFrame(matrix['durations'], index=names, columns=names)

print("Distance Matrix (km):")
print(dist_df.round(2))


Distance Matrix (km):
                   Farm_Punjab  Farm_Haryana  Farm_UP  Warehouse_Punjab  \
Farm_Punjab               0.00        365.15   784.54             88.56   
Farm_Haryana            356.88          0.00   474.42            445.83   
Farm_UP                 789.87        480.43     0.00            878.82   
Warehouse_Punjab         88.86        454.13   873.52              0.00   
Warehouse_Haryana        61.10        321.09   740.47            150.05   
Retailer_Delhi          339.44         34.17   496.68            428.39   
Retailer_UP             548.27        203.20   327.83            637.22   
Retailer_Punjab          98.74        464.01   883.39             81.23   

                   Warehouse_Haryana  Retailer_Delhi  Retailer_UP  \
Farm_Punjab                    61.11          344.70       549.69   
Farm_Haryana                  312.88           35.06       201.97   
Farm_UP                       745.87          500.44       327.08   
Warehouse_Punjab          

In [5]:
# Build Graph using NetworkX
G = nx.DiGraph()

for i, source in enumerate(names):
    for j, target in enumerate(names):
        if i != j:
            G.add_edge(source, target,
                       distance=dist_df.loc[source, target],
                       time=time_df.loc[source, target])


In [6]:
# Shortest path from Farm_Punjab to Retailer_UP based on distance
shortest_path = nx.shortest_path(G, source="Farm_Punjab", target="Retailer_UP", weight="distance")
total_dist = nx.shortest_path_length(G, source="Farm_Punjab", target="Retailer_UP", weight="distance")

print("Shortest Path (by Distance):", shortest_path)
print("Total Distance:", total_dist, "km")


Shortest Path (by Distance): ['Farm_Punjab', 'Retailer_UP']
Total Distance: 549.69 km


In [7]:
import folium

# Initialize Map
m = folium.Map(location=[29.0, 77.0], zoom_start=6)

# Add markers
for name, coord in locations.items():
    folium.Marker(location=coord[::-1], popup=name).add_to(m)

# Draw shortest path
path_coords = [locations[loc] for loc in shortest_path]
folium.PolyLine(locations=[(lat, lon) for lon, lat in path_coords], color="blue", weight=4).add_to(m)

# Show map
m.save("route_map.html")
